In [1]:
# to save as tif in google drive
!pip install imagecodecs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 16.3 MB/s eta 0:00:00


In [2]:
# ML model for cell segmentation
!pip install stardist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.4 MB/s eta 0:00:00


In [3]:
from stardist.models import StarDist2D 
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize

In [4]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, io, img_as_ubyte
from skimage.color import rgb2hed, hed2rgb, rgb2gray
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops_table
from skimage.segmentation import find_boundaries, mark_boundaries, clear_border
import pandas as pd

from glob import glob
import math

In [5]:
from google.colab.patches import cv2_imshow
import cv2

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [7]:
work_dir = '/content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/'          #you might need to change it in your case, look at your folder structure
img_dir = work_dir + 'images_1_CROPPED/'
auto_dir = work_dir + 'images_1_AUTO/alfa/'

In [8]:
images = glob(img_dir+"*_SELECTED_MODEL*")                                #select only images containing SELECTED keyword in it

In [10]:
# Separate the individual stains from the IHC image
def color_separate(ihc_rgb):

    #Convert the RGB image to HED using the prebuilt skimage method
    ihc_hed = rgb2hed(ihc_rgb)
    
    # Create an RGB image for each of the separated stains
    #Convert them to ubyte for easy saving to drive as an image
    null = np.zeros_like(ihc_hed[:, :, 0])
    ihc_h = img_as_ubyte(hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1)))
    ihc_e = img_as_ubyte(hed2rgb(np.stack((null, ihc_hed[:, :, 1], null), axis=-1)))
    ihc_d = img_as_ubyte(hed2rgb(np.stack((null, null, ihc_hed[:, :, 2]), axis=-1)))

    #Optional fun exercise of combining H and DAB stains into a single image with fluorescence look
    
    h = rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1),
                          in_range=(0, np.percentile(ihc_hed[:, :, 0], 99)))
    d = rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1),
                          in_range=(0, np.percentile(ihc_hed[:, :, 2], 99)))

# Cast the two channels into an RGB image, as the blue and green channels
#Convert to ubyte for easy saving as image to local drive
    zdh = img_as_ubyte(np.dstack((null, d, h))) #DAB in green and H in Blue

    return (ihc_h, ihc_e, ihc_d, zdh)

In [11]:
# prints a list of available models 
StarDist2D.from_pretrained()
model = StarDist2D.from_pretrained('2D_versatile_he')
# model_2 = StarDist2D.from_pretrained('2D_versatile_fluo')
# model_3 = StarDist2D.from_pretrained('2D_paper_dsb2018')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_he' for 'StarDist2D'.
5294730/5294730 [==============================] - 0s 0us/step
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [12]:
def variance_of_laplacian(image):
	# compute the Laplacian of the image and then return the focus
	# measure, which is simply the variance of the Laplacian
	return cv2.Laplacian(image, cv2.CV_64F).var()

In [13]:
def gamma_adjust(img):
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

  # compute gamma = log(mid*255)/log(mean)
  mid = 0.5
  mean = np.mean(gray)
  gamma = math.log(mid*255)/math.log(mean)
  print(gamma)

  # do gamma correction
  img_gamma1 = np.power(img, gamma).clip(0,255).astype(np.uint8)
  return img_gamma1

In [14]:
stats = pd.DataFrame(columns = ['img', 'label', 'area'])
save_dir = auto_dir
for img in images:
    ihc_rgb=io.imread(img)
    ihc_rgb = gamma_adjust(ihc_rgb)
    if ihc_rgb.shape[-1] == 4:
        ihc_rgb = ihc_rgb[:, :, :3]
    H,E,D,HD = color_separate(ihc_rgb)
    # H = rgb2gray(H)
    H_labels, H_details = model.predict_instances(normalize(H))
    ################################################# border
    H_labels = clear_border(H_labels)
    ####################################################
    mask = H_labels > 0
    marked = mark_boundaries(ihc_rgb, H_labels, color=(1,0,0))
    fname = save_dir+img.split('/')[-1][:-4]+"_SEGMENTED.jpg"
    io.imsave(fname, marked)
    maskname = save_dir+img.split('/')[-1][:-4]+"_MASK.tif"
    io.imsave(maskname, mask)
    H_props = regionprops_table(H_labels, H, 
                          properties=['label',
                                      'area'])
    H_analysis_results = pd.DataFrame(H_props)
    H_analysis_results['img'] = img.split('/')[-1]
    ######################################### small
    area_threshold = 200
    H_analysis_results = H_analysis_results[H_analysis_results['area'] > area_threshold]
    #########################################
    ######################################### blurry
    # small_images = []
    # for i in range(H_analysis_results.shape[0]):
    #   miny = max(H_analysis_results['bbox-1'].iloc[i] - 10, 0)
    #   minx = max(H_analysis_results['bbox-0'].iloc[i] - 10, 0)
    #   maxy = H_analysis_results['bbox-3'].iloc[i] + 10
    #   maxx = H_analysis_results['bbox-2'].iloc[i] + 10
    #   crop = H[minx:maxx, miny:maxy, :]
    #   if len(list(list(crop)[0])) > 0:
    #     small_images.append(crop)
    # fms = []
    # for i in range(len(small_images)):
    #   # print(i)
    #   gray = cv2.cvtColor(small_images[i], cv2.COLOR_BGR2GRAY)
    #   fm = variance_of_laplacian(gray)
    #   fms.append(fm)
    #   if fm < 65:
    #     H_analysis_results.iloc[i]['label'] = 'None'
    # H_analysis_results = H_analysis_results[H_analysis_results['label'] != 'None']
    ###########################################
    stats = pd.concat([stats, H_analysis_results])
stats.to_excel(save_dir+'measurements_from_stardist.xlsx', index=False)

0.9732789786459246
1/1 [==============================] - 3s 3s/step


<ipython-input-14-d39592c83035> (19): /content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_AUTO/alfa/6_L-08 FFPE Exp 12 ms Gain 1 Satur 100_crop_1_SELECTED_MODEL_MASK.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.


1.0268345431413
1/1 [==============================] - 2s 2s/step


<ipython-input-14-d39592c83035> (19): /content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_AUTO/alfa/8_L04-2 Cryo Exp 15 ms Gain 1 Satur 100_crop_1_SELECTED_MODEL_MASK.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.


0.9885208120108107
1/1 [==============================] - 1s 732ms/step


...

<ipython-input-14-d39592c83035> (19): /content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_AUTO/alfa/33_L27-1 Cryo Exp 23 ms Gain 1 Satur 100_crop_3_SELECTED_MODEL_MASK.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.


1.0245192106277285
1/1 [==============================] - 1s 986ms/step


<ipython-input-14-d39592c83035> (19): /content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_AUTO/alfa/34_L27-1 FFPE Exp 15 ms Gain 1 Satur 100_crop_2_SELECTED_MODEL_MASK.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.


1.0058396371495808
1/1 [==============================] - 2s 2s/step


<ipython-input-14-d39592c83035> (19): /content/drive/MyDrive/0_Lymphoma_PPFE_vs_Cryo/images_1_AUTO/alfa/35_L27-1 FFPE Exp 16 ms Gain 1 Satur 100_crop_3_SELECTED_MODEL_MASK.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
